# Practice notebook for confidence intervals using NHANES data

[github link](https://github.com/karimkmafifi/Inferential-Statistical-Analysis-with-Python---Coursera/blob/master/nhanes_confidence_intervals_practice.ipynb)

In [16]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.api as sm
import scipy.stats

da = pd.read_csv("nhanes_2015_2016.csv")

## Question 1

Restrict the sample to women between 35 and 50, then use the marital status variable [DMDMARTL](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#DMDMARTL) to partition this sample into two groups - women who are currently married, and women who are not currently married.  Within each of these groups, calculate the proportion of women who have completed college.  Calculate 95% confidence intervals for each of these proportions.

## My answer

In [22]:
da['DMDMARTL'] = da['DMDMARTL'].replace({1:'married', 2:'widowed', 
                                         3:'divorced', 4:'separated',
                                        5:'never married', 6:'living with partner',
                                        77:'refused', 99:"don't know", '.':'missing'})

da['DMDMARTL'].unique()

array(['married', 'divorced', 'living with partner', 'separated',
       'never married', nan, 'widowed', 'refused'], dtype=object)

In [23]:
da['RIAGENDR'] = da['RIAGENDR'].replace({1:'male', 2:'female', '.':'missing'})
da['RIAGENDR'].unique()

array(['male', 'female'], dtype=object)

In [5]:
da.columns

Index(['SEQN', 'ALQ101', 'ALQ110', 'ALQ130', 'SMQ020', 'RIAGENDR', 'RIDAGEYR',
       'RIDRETH1', 'DMDCITZN', 'DMDEDUC2', 'DMDMARTL', 'DMDHHSIZ', 'WTINT2YR',
       'SDMVPSU', 'SDMVSTRA', 'INDFMPIR', 'BPXSY1', 'BPXDI1', 'BPXSY2',
       'BPXDI2', 'BMXWT', 'BMXHT', 'BMXBMI', 'BMXLEG', 'BMXARML', 'BMXARMC',
       'BMXWAIST', 'HIQ210'],
      dtype='object')

In [25]:
# Get the women between 35 and 50. The value of RIAGENDR is equal to 2 if a subject is female
middle_age_women = da.query("RIDAGEYR >= 35 & RIDAGEYR <= 50 & RIAGENDR == 'female'")
middle_age_women.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
4,83736,2.0,1.0,1.0,2,female,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0
34,83799,NaN,NaN,NaN,2,female,37,2,1.0,4.0,...,110.0,72.0,66.6,161.6,25.5,NaN,NaN,NaN,NaN,2.0
50,83828,1.0,NaN,2.0,2,female,39,1,2.0,3.0,...,100.0,62.0,71.3,162.0,27.2,36.8,34.6,29.1,94.6,NaN
52,83832,2.0,1.0,4.0,2,female,50,1,2.0,1.0,...,NaN,NaN,105.9,157.7,42.6,29.2,35.0,40.7,129.1,NaN
55,83837,2.0,2.0,NaN,2,female,45,1,1.0,2.0,...,114.0,68.0,77.5,148.3,35.2,30.5,34.0,34.4,107.6,2.0


In [26]:
# Check that our data are correct! This step is not necessary but it is good practice :)
assert np.all(middle_age_women['RIAGENDR'] == 'female')
assert np.all(middle_age_women['RIDAGEYR'] <= 50)
assert np.all(middle_age_women['RIDAGEYR'] >= 35)

# assert လုပ်တာကမှန်/မမှန် စစ်တာ။ မမှန်ရင် AssertionError တက်မည်။

In [28]:
# Partition the group into married and non-married women
married_middle_age_women = middle_age_women.query("DMDMARTL == 'married'")
non_married_middle_age_women = middle_age_women.query("DMDMARTL != 'married'")

# More data checks!
assert np.all(married_middle_age_women['DMDMARTL'] == "married")
assert np.all(non_married_middle_age_women['DMDMARTL'] != "married")

In [29]:
# We now have checked our data is valid and is partitioned correctly. We 
# now can compute the 90% confidence intervals for the proportion which have
# completed college. This is coded in the DMDEDUC2 variable and it is equal to
# 5 if they have completed college or above
married_and_completed_college = married_middle_age_women['DMDEDUC2'] == 5
p_hat_married = np.mean(married_and_completed_college)
married_sample_size = married_and_completed_college.size
"The proportion of married middle age women (N={}) who completed college is: {:.2f}".\
format(married_sample_size, p_hat_married)

'The proportion of married middle age women (N=449) who completed college is: 0.36'

In [30]:
# We can do the same for non-married women
non_married_and_completed_college = non_married_middle_age_women['DMDEDUC2'] == 5
p_hat_non_married = np.mean(non_married_and_completed_college)
non_married_sample_size = non_married_and_completed_college.size
"The proportion of non-married middle age women (N={}) who completed college is: {:.2f}" \
.format(non_married_sample_size, p_hat_non_married)

'The proportion of non-married middle age women (N=338) who completed college is: 0.21'

In [31]:
# We can now compute the confidence intervals. Remember, for a two-sided 
# confidence interval, we need 5% in each of the tails and 95% PPF will give us
# this value !
z_multiplier = scipy.stats.norm.ppf(q = 0.95)
print(z_multiplier)
married_standard_error = np.sqrt(p_hat_married * (1 - p_hat_married) / married_sample_size)
ci_married_lower_bound = p_hat_married - z_multiplier * married_standard_error
ci_married_upper_bound = p_hat_married + z_multiplier * married_standard_error
"A 90% confidence interval for the proportion of married women who completed college is ({:.2f}, {:.2f})".format(
    ci_married_lower_bound, 
    ci_married_upper_bound
)

# ppf = Percent point function (inverse of cdf — percentiles).

1.6448536269514722


'A 90% confidence interval for the proportion of married women who completed college is (0.32, 0.40)'

In [33]:
# We now can do the same for non-married women
z_multiplier = scipy.stats.norm.ppf(q = 0.95)
non_married_standard_error = np.sqrt(p_hat_non_married * (1 - p_hat_non_married) / non_married_sample_size)
ci_non_married_lower_bound = p_hat_non_married - z_multiplier * non_married_standard_error
ci_non_married_upper_bound = p_hat_non_married + z_multiplier * non_married_standard_error
"A 90% confidence interval for the proportion of non-married women who completed college is ({:.2f}, {:.2f})" \
.format(ci_non_married_lower_bound, ci_non_married_upper_bound)

'A 90% confidence interval for the proportion of non-married women who completed college is (0.18, 0.25)'

__Q1a.__ Identify which of the two confidence intervals is wider, and explain why this is the case. 

The width of the confidence interval for the population proportion of married women who completed college is eight percentage points wide while the confidence interval for non-married women is seven points wide. The confidence interval for married women is larger, despite having a larger sample size, because the $\hat{p}_{married}$ estimate is closer to 50% than the $\hat{p}_{non-married}$ so the standard error of $\hat{p}_{married}$ is wider.

__Q1b.__ Write 1-2 sentences summarizing these findings for an audience that does not know what a confidence interval is (the goal here is to report the substance of what you learned about how marital status and educational attainment are related, not to teach a person what a confidence interval is).

We can see that, on average, a middle age woman who is married is more likely to have completed college than a middle age women who is not married, and the confidence intervals for these two estimates do not overlap.

## Question 2

Construct a 95% confidence interval for the proportion of smokers who are female. Construct a 95% confidence interval for the proportion of smokers who are male. Construct a 95% confidence interval for the **difference** between those two gender proportions.

In [14]:
da["SMQ020x"]= da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

In [15]:
dx = da[["SMQ020x", "RIAGENDRx"]].dropna()
dx

,SMQ020x,RIAGENDRx
0,Yes,male
1,Yes,male
2,Yes,male
3,No,female
4,No,female
...,...,...
5730,Yes,female
5731,No,male
5732,Yes,female
5733,Yes,male


In [16]:
dy = pd.crosstab(columns=dx.RIAGENDRx, index=dx.SMQ020x)
dy

RIAGENDRx,female,male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [17]:
dz = dx.groupby(dx.RIAGENDRx).agg({"SMQ020x": [lambda x: np.mean(x=="Yes"), np.size]})
dz.columns = ["Proportion", "Count"]
dz

,Proportion,Count
RIAGENDRx,,
female,0.304845,2972
male,0.513258,2753


In [18]:
female_p = dz.Proportion.female
male_p = dz.Proportion.male
female_n = dz.Count.female
male_n = dz.Count.male
print(female_n, male_n)

2972 2753


In [19]:
# standard error (female)
female_se = np.sqrt(female_p * (1 - female_p) / female_n)
print("Standard error (female) =", female_se)

# standard error (male)
male_se = np.sqrt(male_p * (1 - male_p) / male_n)
print("Standard error (female) =", male_se)

Standard error (female) = 0.008444152146214435
Standard error (female) = 0.009526078653689868


In [20]:
# for female
female_lcb = female_p - 1.96 * female_se
female_ucb = female_p + 1.96 * female_se
print(f"LCB (female) = {female_lcb} and UCB (female) = {female_ucb}")

# for male
male_lcb = male_p - 1.96 * male_se
male_ucb = male_p + 1.96 * male_se
print(f"LCB (male) = {male_lcb} and UCB (male) = {male_ucb}")

LCB (female) = 0.288294683866098 and UCB (female) = 0.32139576027925865
LCB (male) = 0.49458714955108174 and UCB (male) = 0.531929377873546


**Using Statsmodels**

In [21]:
# for female smoker
lcb, ucb = sm.stats.proportion_confint(count=dy.female.Yes , nobs=dy.female.Yes + dy.female.No)
lcb, ucb

(0.2882949879861214, 0.32139545615923526)

In [22]:
# for male smoker
lcb, ucb = sm.stats.proportion_confint(count=dy.male.Yes , nobs=dy.male.Yes + dy.male.No)
lcb, ucb

(0.49458749263718593, 0.5319290347874418)

In [23]:
diff_se = np.sqrt(female_se**2 + male_se**2)
print(diff_se)

0.012729881381407434


In [24]:
# male proportion - female proportion
diff_p = dz.Proportion.male - dz.Proportion.female
lcb = diff_p - 2  * diff_se
ucb = diff_p + 2 * diff_se
print(f"LCB = {lcb} and UCB = {ucb}")

LCB = 0.18295327887682067 and UCB = 0.2338728044024504


In [25]:
# female proportion - male proportion
diff_p = dz.Proportion.female - dz.Proportion.male
lcb = diff_p - 2  * diff_se
ucb = diff_p + 2 * diff_se
print(f"LCB = {lcb} and UCB = {ucb}")

LCB = -0.2338728044024504 and UCB = -0.18295327887682067


__Q2a.__ Why might it be relevant to report the separate gender proportions **and** the difference between the gender proportions?

__Q2b.__ How does the **width** of the confidence interval for the difference between the gender proportions compare to the widths of the confidence intervals for the separate gender proportions?

## Question 3

Construct a 95% interval for height ([BMXHT](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/BMX_I.htm#BMXHT)) in centimeters.  Then convert height from centimeters to inches by dividing by 2.54, and construct a 95% confidence interval for height in inches.  Finally, convert the endpoints (the lower and upper confidence limits) of the confidence interval from inches to back to centimeters.

In [26]:
da["BMXHTi"] = da["BMXHT"] / 2.54
da["BMXHTi"].dropna(inplace=True)

In [27]:
mean_h = np.mean(da["BMXHTi"])
std_h = np.std(da["BMXHTi"])
count_h = len(da["BMXHTi"])

lcb = mean_h - 1.96 * (std_h / np.sqrt(count_h))
ucb = mean_h + 1.96 * (std_h / np.sqrt(count_h))
print(f"LCB (inches) = {lcb} and UCB (inches) = {ucb}")

LCB (inches) = 65.30787049524722 and UCB (inches) = 65.51325901586426


In [28]:
print(f"LCB (cm) = {lcb * 2.54} and UCB (cm) = {ucb * 2.54}")

LCB (cm) = 165.88199105792793 and UCB (cm) = 166.40367790029524


__Q3a.__ Describe how the confidence interval constructed in centimeters relates to the confidence interval constructed in inches.

## Question 4

Partition the sample based on 10-year age bands, i.e. the resulting groups will consist of people with ages from 18-28, 29-38, etc. Construct 95% confidence intervals for the difference between the mean BMI for females and for males within each age band.

In [52]:
da["RIDAGEYR"].min(), da["RIDAGEYR"].max()

(18, 80)

In [73]:
bins = np.arange(start=da["RIDAGEYR"].min(), stop=da["RIDAGEYR"].max()+10, step=10)
da["age_band"] = pd.cut(x=da["RIDAGEYR"], bins=bins, right=True)

In [74]:
age_gp = da.groupby(["age_band","RIAGENDRx"]).agg({"BMXBMI": [np.mean, np.std, np.size]}).unstack()
age_gp

BMXBMI                                           
                mean                  std             size     
RIAGENDRx     female       male    female      male female male
age_band                                                       
(18, 28]   28.019433  27.058186  8.048854  6.679515    498  458
(28, 38]   29.943443  29.697180  7.959097  6.726690    494  467
(38, 48]   31.003733  29.514646  8.044642  6.104950    514  398
(48, 58]   30.787361  29.385132  7.647590  6.151534    454  419
(58, 68]   31.054664  29.232462  7.779502  5.959024    466  470
(68, 78]   30.537818  28.720270  6.780588  5.336652    279  307
(78, 88]   27.850000  27.464368  5.483781  4.695650    201  177

In [75]:
age_gp["BMXBMI","sem","female"] = age_gp["BMXBMI","std","female"] / np.sqrt(age_gp["BMXBMI","size","female"])
age_gp["BMXBMI","sem","male"] = age_gp["BMXBMI","std","male"] / np.sqrt(age_gp["BMXBMI","size","male"])

In [76]:
age_gp

BMXBMI                                                       \
                mean                  std             size            sem   
RIAGENDRx     female       male    female      male female male    female   
age_band                                                                    
(18, 28]   28.019433  27.058186  8.048854  6.679515    498  458  0.360678   
(28, 38]   29.943443  29.697180  7.959097  6.726690    494  467  0.358097   
(38, 48]   31.003733  29.514646  8.044642  6.104950    514  398  0.354834   
(48, 58]   30.787361  29.385132  7.647590  6.151534    454  419  0.358919   
(58, 68]   31.054664  29.232462  7.779502  5.959024    466  470  0.360378   
(68, 78]   30.537818  28.720270  6.780588  5.336652    279  307  0.405943   
(78, 88]   27.850000  27.464368  5.483781  4.695650    201  177  0.386796   

                     
                     
RIAGENDRx      male  
age_band             
(18, 28]   0.312113  
(28, 38]   0.311274  
(38, 48]   0.306014  
(48, 58]   0.300522  
(58, 68]   0.274869  
(68, 78]   0.304579  
(78, 88]   0.352947

In [77]:
age_gp["BMXBMI","mean_diff",""] = age_gp["BMXBMI","mean","male"] - age_gp["BMXBMI","mean","female"]
age_gp["BMXBMI","sem_diff",""] = np.sqrt(age_gp["BMXBMI","sem","male"]**2 + age_gp["BMXBMI","sem","female"]**2)  
age_gp["BMXBMI","lcb",""] = age_gp["BMXBMI","mean_diff",""] - 1.96 * age_gp["BMXBMI","sem_diff",""]
age_gp["BMXBMI","ucb",""] = age_gp["BMXBMI","mean_diff",""] + 1.96 * age_gp["BMXBMI","sem_diff",""]

In [78]:
age_gp

BMXBMI                                                       \
                mean                  std             size            sem   
RIAGENDRx     female       male    female      male female male    female   
age_band                                                                    
(18, 28]   28.019433  27.058186  8.048854  6.679515    498  458  0.360678   
(28, 38]   29.943443  29.697180  7.959097  6.726690    494  467  0.358097   
(38, 48]   31.003733  29.514646  8.044642  6.104950    514  398  0.354834   
(48, 58]   30.787361  29.385132  7.647590  6.151534    454  419  0.358919   
(58, 68]   31.054664  29.232462  7.779502  5.959024    466  470  0.360378   
(68, 78]   30.537818  28.720270  6.780588  5.336652    279  307  0.405943   
(78, 88]   27.850000  27.464368  5.483781  4.695650    201  177  0.386796   

                                                             
                    mean_diff  sem_diff       lcb       ucb  
RIAGENDRx      male                                          
age_band                                                     
(18, 28]   0.312113 -0.961247  0.476973 -1.896114 -0.026381  
(28, 38]   0.311274 -0.246263  0.474473 -1.176230  0.683705  
(38, 48]   0.306014 -1.489086  0.468563 -2.407470 -0.570703  
(48, 58]   0.300522 -1.402230  0.468120 -2.319745 -0.484714  
(58, 68]   0.274869 -1.822202  0.453239 -2.710551 -0.933853  
(68, 78]   0.304579 -1.817548  0.507502 -2.812252 -0.822844  
(78, 88]   0.352947 -0.385632  0.523624 -1.411936  0.640672

__Q4a.__ How do the widths of these confidence intervals differ?  Provide an explanation for any substantial differences in the confidence interval widths that you see.

## Question 5

Construct a 95% confidence interval for the first and second systolic blood pressure measures, and for the difference between the first and second systolic blood pressure measurements within a subject.

In [89]:
sys_mean1 = da["BPXSY1"].mean()
sys_std1 = da["BPXSY1"].std()
sys_n1 = len(da["BPXSY1"])
sys_sem1 = sys_std1 / np.sqrt(sys_n1)
lcb1 = sys_mean1 - 1.96 * sys_sem1
ucb1 = sys_mean1 + 1.96 * sys_sem1

print(f"LCB = {lcb1} and UCB = {ucb1}")

LCB = 124.60630134587866 and UCB = 125.56292657487676


In [86]:
sys_mean2 = da["BPXSY2"].mean()
sys_std2 = da["BPXSY2"].std()
sys_n2 = len(da["BPXSY2"])
sys_sem2 = sys_std2 / np.sqrt(sys_n2)
lcb2 = sys_mean2 - 1.96 * sys_sem2
ucb2 = sys_mean2 + 1.96 * sys_sem2

print(f"LCB = {lcb2} and UCB = {ucb2}")

LCB = 124.30351040092819 and UCB = 125.26252392608174


In [90]:
sys_mean_diff = sys_mean1 - sys_mean2
sys_sem_diff = np.sqrt(sys_sem1**2 + sys_sem2**2)
lcb_diff = sys_mean_diff - 1.96 * sys_sem_diff
ucb_diff = sys_mean_diff + 1.96 * sys_sem_diff

print(f"LCB = {lcb_diff} and UCB = {ucb_diff}")

LCB = -0.37568430617515414 and UCB = 0.9788778999206417


__Q5a.__ Based on these confidence intervals, would you say that a difference of zero between the population mean values of the first and second systolic blood pressure measures is consistent with the data?

__Q5b.__ Discuss how the width of the confidence interval for the within-subject difference compares to the widths of the confidence intervals for the first and second measures.

## Question 6

Construct a 95% confidence interval for the mean difference between the average age of a smoker, and the average age of a non-smoker.

In [96]:
dx = da.groupby("SMQ020x").agg({"RIDAGEYR":[np.mean, np.std, np.size]})
dx.columns = ["Mean", "Std", "Num"]
dx

,Mean,Std,Num
SMQ020x,,,
No,45.259836,18.543286,3406
Yes,52.096593,17.461141,2319


In [99]:
smkr_se = dx.Std.Yes / np.sqrt(dx.Num.Yes)
nsmkr_se = dx.Std.No / np.sqrt(dx.Num.No)

mean_diff = dx.Mean.Yes - dx.Mean.No
se_diff = np.sqrt(smkr_se**2 + nsmkr_se**2)

lcb = mean_diff - 1.96 * se_diff
ucb = mean_diff + 1.96 * se_diff

print(f"LCB (smoker) = {lcb} and UCB (smoker) = {ucb}")

LCB (smoker) = 5.891821038686713 and UCB (smoker) = 7.781694511200259


__Q6a.__ Use graphical and numerical techniques to compare the variation in the ages of smokers to the variation in the ages of non-smokers.  

__Q6b.__ Does it appear that uncertainty about the mean age of smokers, or uncertainty about the mean age of non-smokers contributed more to the uncertainty for the mean difference that we are focusing on here?